In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# 데이터 경로 설정
train_dir = 'coco2017/train2017'
val_dir = 'coco2017/val2017'

# 데이터 증강 및 로드
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.2)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='binary')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=32, class_mode='binary')

# MobileNetV2 모델 로드
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델 구성
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# 일부 레이어를 학습 가능하게 설정
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(train_generator, validation_data=val_generator, epochs=10)

# 모델 저장
model.save('walking_running_classifier.h5')


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import mediapipe as mp


# 모델 로드
model = load_model('walking_running_classifier.h5')

# MediaPipe 포즈 추정 설정
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 웹캠 피드 설정
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 포즈 추정
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    if results.pose_landmarks:
        # 랜드마크 추출
        landmarks = results.pose_landmarks.landmark
        feature_vector = []
        for lm in landmarks:
            feature_vector.append(lm.x)
            feature_vector.append(lm.y)
            feature_vector.append(lm.z)
        
        # 모델 입력 준비
        feature_vector = np.array(feature_vector).reshape(1, -1)
        
        # 예측
        prediction = model.predict(feature_vector)
        label = "walking" if prediction < 0.5 else "running"

        # 결과 화면에 표시
        cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Webcam Feed', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
